In [46]:
from pathlib import Path
import json
import pandas as pd

LOG_PATH = Path("/home/moon/Desktop/Financial_Document_Analyzer/data/logs/financebench_eval_20251127_163151.jsonl")
assert LOG_PATH.exists(), f"Log file not found: {LOG_PATH}"

rows = []
with LOG_PATH.open("r", encoding="utf-8") as fh:
    for line in fh:
        line = line.strip()
        if not line:
            continue
        rows.append(json.loads(line))
df = pd.DataFrame(rows)
df

,doc_name,question,ground_truth,question_type,question_reasoning,answer,citations,hits,eval_is_same,eval_reasoning
0,AMCOR_2022_8K_dated-2022-07-01,What was the key agenda of the AMCOR's 8k fili...,"Amcor Finance (USA), Inc. and Amcor Flexibles ...",novel-generated,None,The key agenda of AMCOR's 8K filing dated 1st ...,"[{'i': 5, 'source_doc': 'AMCOR_2022_8K_dated-2...","[{'chunk_id': '28', 'source_doc': 'AMCOR_2022_...",True,Both answers describe the same key agenda: the...
1,AMERICANEXPRESS_2022_10K,Which debt securities are registered to trade ...,There are none,domain-relevant,Information extraction,Not reported in the filing.,"[{'i': 10, 'source_doc': 'AMERICANEXPRESS_2022...","[{'chunk_id': '943', 'source_doc': 'AMERICANEX...",False,Answer 1 states that no debt securities exist ...
2,AMERICANEXPRESS_2022_10K,What are the geographies that American Express...,"United States, EMEA, APAC, and LACC",domain-relevant,Information extraction,American Express primarily operates in the Uni...,"[{'i': 2, 'source_doc': 'AMERICANEXPRESS_2022_...","[{'chunk_id': '1197', 'source_doc': 'AMERICANE...",False,Answer 2 does not specify the four region cate...
3,AMERICANEXPRESS_2022_10K,Does AMEX have an improving operating margin p...,Performance is not measured through operating ...,domain-relevant,Numerical reasoning OR information extraction,Not reported in the filing.,"[{'i': 8, 'source_doc': 'AMERICANEXPRESS_2022_...","[{'chunk_id': '492', 'source_doc': 'AMERICANEX...",False,Answer 1 says that performance is not measured...
4,AMERICANEXPRESS_2022_10K,What drove gross margin change as of the FY202...,Performance is not measured through gross margin,domain-relevant,Logical reasoning (based on numerical reasonin...,Gross margin is not reported in the filing. Th...,"[{'i': 6, 'source_doc': 'AMERICANEXPRESS_2022_...","[{'chunk_id': '492', 'source_doc': 'AMERICANEX...",True,Both responses state that gross margin is not ...
5,AMERICANEXPRESS_2022_10K,How much has the effective tax rate of America...,The effective tax rate for American Express ha...,domain-relevant,Numerical reasoning,The effective tax rate decreased by 3.0 percen...,"[{'i': 1, 'source_doc': 'AMERICANEXPRESS_2022_...","[{'chunk_id': '275', 'source_doc': 'AMERICANEX...",True,Both answers state that American Express’s eff...
6,AMERICANEXPRESS_2022_10K,What was the largest liability in American Exp...,Customer deposits,novel-generated,None,Not reported in the filing.,"[{'i': 4, 'source_doc': 'AMERICANEXPRESS_2022_...","[{'chunk_id': '24', 'source_doc': 'AMERICANEXP...",False,Answer 1 identifies 'Customer deposits' as the...
7,AMERICANEXPRESS_2022_10K,Was American Express able to retain card membe...,Yes,novel-generated,None,No answer,[],"[{'chunk_id': '24', 'source_doc': 'AMERICANEXP...",False,Answer 1 states that American Express was able...
8,BESTBUY_2024Q2_10Q,Was there any drop in Cash & Cash equivalents ...,"Yes, there was a decline of ~42% between FY202...",novel-generated,None,The sources do not provide data for Q2 of FY20...,"[{'i': 7, 'source_doc': 'BESTBUY_2024Q2_10Q', ...","[{'chunk_id': '165', 'source_doc': 'BESTBUY_20...",False,Answer 1 claims a ~42% decline in Cash & Cash ...
9,BESTBUY_2024Q2_10Q,Was there any change in the number of Best Buy...,"Yes, there is decline in number stores by 1.32...",novel-generated,None,There was a decrease of 23 stores between the ...,"[{'i': 2, 'source_doc': 'BESTBUY_2024Q2_10Q', ...","[{'chunk_id': '151', 'source_doc': 'BESTBUY_20...",False,Answer 1 reports a decline from 982 to 969 sto...


In [47]:
preview_cols = ['doc_name', 'question', 'question_type', 'ground_truth', 'answer', 'eval_is_same', 'eval_reasoning', 'citations',]
df_preview = df[preview_cols].copy()
df_preview['question_preview'] = df_preview['question'].astype(str).str.slice(0, 200)
df_preview['answer_preview'] = df_preview['answer'].astype(str).str.slice(0, 200)
df_preview = df_preview[['doc_name', 'question', 'question_type', 'ground_truth', 'answer', 'eval_is_same', 'eval_reasoning']]
html_path = LOG_PATH.with_suffix('.html')
df_preview.to_html(html_path, index=False, escape=False)
print(f'Saved preview table to {html_path}')

Saved preview table to /home/moon/Desktop/Financial_Document_Analyzer/data/logs/financebench_eval_20251127_163151.html


In [48]:
# Basic metrics
total = len(df)
correct = int(df['eval_is_same'].sum())
accuracy = correct / total if total else 0
print(f'Total questions: {total}')
print(f'Correct: {correct} ({accuracy:.2%})')
print(f'Wrong: {total - correct}')

print('\nAccuracy by question_type:')
grouped = df.groupby('question_type')['eval_is_same'].agg(['mean', 'count'])
display(grouped.rename(columns={'mean': 'accuracy'}))

Total questions: 30
Correct: 12 (40.00%)
Wrong: 18

Accuracy by question_type:


,accuracy,count
question_type,,
domain-relevant,0.500,12
metrics-generated,0.000,2
novel-generated,0.375,16


In [49]:
# Inspect incorrect predictions
bad_cases = df[df['eval_is_same'] == False]
print(f'Bad cases: {len(bad_cases)}')
bad_cases[['question', 'question_type', 'answer', 'ground_truth']]


Bad cases: 18


,question,question_type,answer,ground_truth
1,Which debt securities are registered to trade ...,domain-relevant,Not reported in the filing.,There are none
2,What are the geographies that American Express...,domain-relevant,American Express primarily operates in the Uni...,"United States, EMEA, APAC, and LACC"
3,Does AMEX have an improving operating margin p...,domain-relevant,Not reported in the filing.,Performance is not measured through operating ...
6,What was the largest liability in American Exp...,novel-generated,Not reported in the filing.,Customer deposits
7,Was American Express able to retain card membe...,novel-generated,No answer,Yes
8,Was there any drop in Cash & Cash equivalents ...,novel-generated,The sources do not provide data for Q2 of FY20...,"Yes, there was a decline of ~42% between FY202..."
9,Was there any change in the number of Best Buy...,novel-generated,There was a decrease of 23 stores between the ...,"Yes, there is decline in number stores by 1.32..."
12,Has Boeing reported any materially important o...,domain-relevant,Not reported in the filing.,Yes. Multiple lawsuits have been filed against...
13,Does Boeing have an improving gross margin pro...,domain-relevant,Not reported in the filing. The provided sourc...,Yes. Boeing has an improving gross margin prof...
14,Who are the primary customers of Boeing as of ...,domain-relevant,No answer,Boeing's primary customers as of FY2022 are a ...


In [50]:
# Inspect a specific bad case (default: first failure)
idx = bad_cases.index[0] if len(bad_cases) else None
if idx is not None:
    rec = bad_cases.loc[idx]
    print('Question:', rec['question'])
    print('\nGround Truth:', rec['ground_truth'])
    print('\nModel Answer:', rec['answer'])
    print('\nCitations:', rec['citations'])
    print('\nHits (top-k chunk excerpts):')
    for hit in rec['hits']:
        print('-', hit.get('doc_id'), '|', (hit.get('text') or ''))
else:
    print('No bad cases available.')


Question: Which debt securities are registered to trade on a national securities exchange under American Express' name as of 2022?

Ground Truth: There are none

Model Answer: Not reported in the filing.

Citations: [{'i': 10, 'source_doc': 'AMERICANEXPRESS_2022_10K', 'chunk_id': '24', 'element': 'text', 'page_start': 5, 'page_end': 5, 'text': 'For the year ended December 31, 2022, worldwide billed business (spending on American Express cards issued by us) was $1,338 billion and at December 31, 2022, we had 76.7 million proprietary cards in force worldwide.'}]

Hits (top-k chunk excerpts):
- AMERICANEXPRESS_2022_10K_p176 | AMERICAN EXPRESS COMPANY AMERICAN EXPRESS NATIONAL BANK
- AMERICANEXPRESS_2022_10K_p252 | American Express Company 56th Street AXP Campus LLC American Express Bank LLC Russian Federation American Express Banking Corp. American Express Travel Related Services Company, Inc. Netherlands Antilles Page 1
- AMERICANEXPRESS_2022_10K_p252 | Country Name Jurisdiction United S

In [51]:
# Inspect correct predictions
good_cases = df[df['eval_is_same'] == True]
print(f'Good cases: {len(good_cases)}')
good_cases[['question', 'question_type', 'answer', 'ground_truth']]

Good cases: 12


,question,question_type,answer,ground_truth
0,What was the key agenda of the AMCOR's 8k fili...,novel-generated,The key agenda of AMCOR's 8K filing dated 1st ...,"Amcor Finance (USA), Inc. and Amcor Flexibles ..."
4,What drove gross margin change as of the FY202...,domain-relevant,Gross margin is not reported in the filing. Th...,Performance is not measured through gross margin
5,How much has the effective tax rate of America...,domain-relevant,The effective tax rate decreased by 3.0 percen...,The effective tax rate for American Express ha...
10,Which Best Buy product category performed the ...,novel-generated,The Entertainment category performed the best ...,The entertainment segment experienced the high...
11,Are there any product categories / service cat...,domain-relevant,"Yes, three product/service categories exceeded...",Yes. Boeing has product and service categories...
16,What production rate changes is Boeing forecas...,novel-generated,Boeing forecasts increasing production rates f...,Boeing forecasts an increase in the production...
18,Which business segment of JnJ will be treated ...,novel-generated,The Consumer Health business segment of Johnso...,The Consumer Health business segment will be t...
19,What is the amount of the gain accruing to JnJ...,novel-generated,The gain accruing to JnJ as a result of the se...,JnJ will make a gain of approximately $20 bill...
20,What is the amount of the cash proceeds that J...,novel-generated,Johnson & Johnson generated $13.2 billion in c...,JnJ realised $13.2 billion in cash proceeds fr...
21,What are the geographies that Pepsico primaril...,domain-relevant,Pepsico primarily operates in the following ge...,"As of FY2022, Pepsico primarily operates in th..."


In [52]:
# Inspect a specific good case (default: first failure)
idx = good_cases.index[1] if len(good_cases) else None
if idx is not None:
    rec = good_cases.loc[idx]
    print('Question:', rec['question'])
    print('\nGround Truth:', rec['ground_truth'])
    print('\nModel Answer:', rec['answer'])
    print('\nCitations:', rec['citations'])
    print('\nHits (top-k chunk excerpts):')
    for hit in rec['hits']:
        print("'chunk_id:'", hit.get('chunk_id'), '|', (hit.get('text') or ''))
else:
    print('No bad cases available.')


Question: What drove gross margin change as of the FY2022 for American Express? If gross margin is not a useful metric for a company like this, then please state that and explain why.

Ground Truth: Performance is not measured through gross margin

Model Answer: Gross margin is not reported in the filing. The provided sources do not mention gross margin or its drivers for FY2022. American Express’ financial disclosures focus on metrics like billed business revenue, credit losses, and operating cash flows rather than gross margin. This suggests that gross margin may not be a standard metric for evaluating the company’s performance, given its spend-centric business model, which prioritizes revenue from card spending over traditional cost-based margin analysis.

Citations: [{'i': 6, 'source_doc': 'AMERICANEXPRESS_2022_10K', 'chunk_id': '519', 'element': 'table', 'page_start': 99, 'page_end': 99, 'text': '$ Adjustments to reconcile net income to net cash provided by operating activities: P